In [1]:
import pandas as pd

## Brands Data Exploration and data cleaning

In [2]:
brands = pd.read_json("brands.json.gz", lines=True, compression="gzip")
brands.head()

,_id,barcode,category,categoryCode,cpg,name,topBrand,brandCode
0,{'$oid': '601ac115be37ce2ead437551'},511111019862,Baking,BAKING,"{'$id': {'$oid': '601ac114be37ce2ead437550'}, ...",test brand @1612366101024,0.0,NaN
1,{'$oid': '601c5460be37ce2ead43755f'},511111519928,Beverages,BEVERAGES,"{'$id': {'$oid': '5332f5fbe4b03c9a25efd0ba'}, ...",Starbucks,0.0,STARBUCKS
2,{'$oid': '601ac142be37ce2ead43755d'},511111819905,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176
3,{'$oid': '601ac142be37ce2ead43755a'},511111519874,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146051,0.0,TEST BRANDCODE @1612366146051
4,{'$oid': '601ac142be37ce2ead43755e'},511111319917,Candy & Sweets,CANDY_AND_SWEETS,"{'$id': {'$oid': '5332fa12e4b03c9a25efd1e7'}, ...",test brand @1612366146827,0.0,TEST BRANDCODE @1612366146827


In [3]:
brands.shape

(1167, 8)

In [4]:
# below columns has missing values
brands.isnull().sum(axis=0)

_id               0
barcode           0
category        155
categoryCode    650
cpg               0
name              0
topBrand        612
brandCode       234
dtype: int64

Practically, every brand must have a category assigned to it, this the data discrepancy and thus must be checked.

In [5]:
brands.columns

Index(['_id', 'barcode', 'category', 'categoryCode', 'cpg', 'name', 'topBrand',
       'brandCode'],
      dtype='object')

In [6]:
# _id and cpg column has values in form of a dictionary thus we need to expand the datasets and these values into distinct columns
# _id has only 1 key value pair
brands_id = pd.json_normalize(brands._id)

In [7]:
brands_id

,$oid
0,601ac115be37ce2ead437551
1,601c5460be37ce2ead43755f
2,601ac142be37ce2ead43755d
3,601ac142be37ce2ead43755a
4,601ac142be37ce2ead43755e
...,...
1162,5f77274dbe37ce6b592e90c0
1163,5dc1fca91dda2c0ad7da64ae
1164,5f494c6e04db711dd8fe87e7
1165,5a021611e4b00efe02b02a57


In [8]:
brands["_id"] = brands_id["$oid"]

In [9]:
# converting the values in cpg columns to diffrent columns
brands["cpg"].apply(pd.Series)

,$id,$ref
0,{'$oid': '601ac114be37ce2ead437550'},Cogs
1,{'$oid': '5332f5fbe4b03c9a25efd0ba'},Cogs
2,{'$oid': '601ac142be37ce2ead437559'},Cogs
3,{'$oid': '601ac142be37ce2ead437559'},Cogs
4,{'$oid': '5332fa12e4b03c9a25efd1e7'},Cogs
...,...,...
1162,{'$oid': '5f77274dbe37ce6b592e90bf'},Cogs
1163,{'$oid': '53e10d6368abd3c7065097cc'},Cogs
1164,{'$oid': '5332fa12e4b03c9a25efd1e7'},Cogs
1165,{'$oid': '5332f5f6e4b03c9a25efd0b4'},Cogs


In [10]:
# normalising the cpg columns
brands_cpg = pd.json_normalize(brands.cpg)

In [11]:
brands_cpg.columns = ["ref", "oid"]

In [12]:
brands["cpg_ref"] = brands_cpg["ref"]
brands["cpg_id_oid"] = brands_cpg["oid"]

In [13]:
brands.drop(["cpg"], axis=1, inplace=True)

In [14]:
brands

,_id,barcode,category,categoryCode,name,topBrand,brandCode,cpg_ref,cpg_id_oid
0,601ac115be37ce2ead437551,511111019862,Baking,BAKING,test brand @1612366101024,0.0,NaN,Cogs,601ac114be37ce2ead437550
1,601c5460be37ce2ead43755f,511111519928,Beverages,BEVERAGES,Starbucks,0.0,STARBUCKS,Cogs,5332f5fbe4b03c9a25efd0ba
2,601ac142be37ce2ead43755d,511111819905,Baking,BAKING,test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176,Cogs,601ac142be37ce2ead437559
3,601ac142be37ce2ead43755a,511111519874,Baking,BAKING,test brand @1612366146051,0.0,TEST BRANDCODE @1612366146051,Cogs,601ac142be37ce2ead437559
4,601ac142be37ce2ead43755e,511111319917,Candy & Sweets,CANDY_AND_SWEETS,test brand @1612366146827,0.0,TEST BRANDCODE @1612366146827,Cogs,5332fa12e4b03c9a25efd1e7
...,...,...,...,...,...,...,...,...,...
1162,5f77274dbe37ce6b592e90c0,511111116752,Baking,BAKING,test brand @1601644365844,NaN,NaN,Cogs,5f77274dbe37ce6b592e90bf
1163,5dc1fca91dda2c0ad7da64ae,511111706328,Breakfast & Cereal,NaN,Dippin Dots® Cereal,NaN,DIPPIN DOTS CEREAL,Cogs,53e10d6368abd3c7065097cc
1164,5f494c6e04db711dd8fe87e7,511111416173,Candy & Sweets,CANDY_AND_SWEETS,test brand @1598639215217,NaN,TEST BRANDCODE @1598639215217,Cogs,5332fa12e4b03c9a25efd1e7
1165,5a021611e4b00efe02b02a57,511111400608,Grocery,NaN,LIPTON TEA Leaves,0.0,LIPTON TEA Leaves,Cogs,5332f5f6e4b03c9a25efd0b4


In [15]:
brands.shape

(1167, 9)

In [16]:
# checking for duplicates
brands_dup = brands.drop_duplicates(
    subset=None, keep="first", inplace=False, ignore_index=False
)
print(brands_dup.shape)

(1167, 9)


no duplicate rows in brands

## Receipts Data Exploration and data cleaning

In [17]:
receipts = pd.read_json("receipts.json.gz", lines=True, compression="gzip")
receipts.head()

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,{'$oid': '5ff1e1eb0a720f0523000575'},500.0,"Receipt number 2 completed, bonus point schedu...",{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687536000},{'$date': 1609687531000},500.0,{'$date': 1609632000000},5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.0,5ff1e1eacfcf6c399c274ae6
1,{'$oid': '5ff1e1bb0a720f052300056b'},150.0,"Receipt number 5 completed, bonus point schedu...",{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687488000},{'$date': 1609687483000},150.0,{'$date': 1609601083000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.0,5ff1e194b6a9d73a3a9f1052
2,{'$oid': '5ff1e1f10a720f052300057a'},5.0,All-receipts receipt bonus,{'$date': 1609687537000},{'$date': 1609687537000},NaN,{'$date': 1609687542000},NaN,5.0,{'$date': 1609632000000},1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.0,5ff1e1f1cfcf6c399c274b0b
3,{'$oid': '5ff1e1ee0a7214ada100056f'},5.0,All-receipts receipt bonus,{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687539000},{'$date': 1609687534000},5.0,{'$date': 1609632000000},4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.0,5ff1e1eacfcf6c399c274ae6
4,{'$oid': '5ff1e1d20a7214ada1000561'},5.0,All-receipts receipt bonus,{'$date': 1609687506000},{'$date': 1609687506000},{'$date': 1609687511000},{'$date': 1609687511000},{'$date': 1609687506000},5.0,{'$date': 1609601106000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.0,5ff1e194b6a9d73a3a9f1052


In [18]:
receipts.shape

(1119, 15)

In [19]:
# checcking for null values in the columns
receipts.isnull().sum(axis=0)

_id                          0
bonusPointsEarned          575
bonusPointsEarnedReason    575
createDate                   0
dateScanned                  0
finishedDate               551
modifyDate                   0
pointsAwardedDate          582
pointsEarned               510
purchaseDate               448
purchasedItemCount         484
rewardsReceiptItemList     440
rewardsReceiptStatus         0
totalSpent                 435
userId                       0
dtype: int64

In [20]:
receipts_col = receipts.columns
receipts_col

Index(['_id', 'bonusPointsEarned', 'bonusPointsEarnedReason', 'createDate',
       'dateScanned', 'finishedDate', 'modifyDate', 'pointsAwardedDate',
       'pointsEarned', 'purchaseDate', 'purchasedItemCount',
       'rewardsReceiptItemList', 'rewardsReceiptStatus', 'totalSpent',
       'userId'],
      dtype='object')

In [21]:
# fucntion to normalize the values in clomuns into distinct columns in a dataframe
def json_normalize(df, col):
    df_col = pd.json_normalize(df[col])
    return df_col


# prints result to see distinct value after normalising the columsn with multiple data in dictionary
dict_of_df = {}
for i in receipts_col:
    print(i)
    df = json_normalize(receipts, i)
    print(df)

_id
                          $oid
0     5ff1e1eb0a720f0523000575
1     5ff1e1bb0a720f052300056b
2     5ff1e1f10a720f052300057a
3     5ff1e1ee0a7214ada100056f
4     5ff1e1d20a7214ada1000561
...                        ...
1114  603cc0630a720fde100003e6
1115  603d0b710a720fde1000042a
1116  603cf5290a720fde10000413
1117  603ce7100a7217c72c000405
1118  603c4fea0a7217c72c000389

[1119 rows x 1 columns]
bonusPointsEarned
Empty DataFrame
Columns: []
Index: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, ...]

[1119 rows x 0 columns]
bonusPointsEarnedReason
Empty DataFrame
Columns: []
Index: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18,

#### rewardsReceiptItemList has more than one values


In [22]:
receipts["rewardsReceiptItemList"].apply(pd.Series)

,0,1,2,3,4,5,6,7,8,9,...,449,450,451,452,453,454,455,456,457,458
0,"{'barcode': '4011', 'description': 'ITEM NOT F...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"{'barcode': '4011', 'description': 'ITEM NOT F...","{'barcode': '028400642255', 'description': 'DO...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"{'needsFetchReview': False, 'partnerItemId': '...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"{'barcode': '4011', 'description': 'ITEM NOT F...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"{'barcode': '4011', 'description': 'ITEM NOT F...","{'barcode': '1234', 'finalPrice': '2.56', 'ite...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1114,"{'barcode': 'B076FJ92M4', 'description': 'muel...","{'barcode': 'B07BRRLSVC', 'description': 'thin...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1115,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1116,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1117,"{'barcode': 'B076FJ92M4', 'description': 'muel...","{'barcode': 'B07BRRLSVC', 'description': 'thin...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
receipts1 = receipts.explode("rewardsReceiptItemList")
receipts1.reset_index(inplace=True)

In [24]:
receipts_rewardsReceiptItemList = json_normalize(receipts1, "rewardsReceiptItemList")
receipts_rewardsReceiptItemList.isnull().sum(axis=0)

barcode                               4291
description                            821
finalPrice                             614
itemPrice                              614
needsFetchReview                      6568
partnerItemId                          440
preventTargetGapPoints                7023
quantityPurchased                      614
userFlaggedBarcode                    7044
userFlaggedNewItem                    7058
userFlaggedPrice                      7082
userFlaggedQuantity                   7082
needsFetchReviewReason                7162
pointsNotAwardedReason                7041
pointsPayerId                         6114
rewardsGroup                          5650
rewardsProductPartnerId               5112
userFlaggedDescription                7176
originalMetaBriteBarcode              7310
originalMetaBriteDescription          7371
brandCode                             4781
competitorRewardsGroup                7106
discountedItemPrice                   1612
originalRec

In [25]:
# expanded the rewardsReceiptItemList columns
receipts_rewardsReceiptItemList

,barcode,description,finalPrice,itemPrice,needsFetchReview,partnerItemId,preventTargetGapPoints,quantityPurchased,userFlaggedBarcode,userFlaggedNewItem,...,itemNumber,originalMetaBriteQuantityPurchased,pointsEarned,targetPrice,competitiveProduct,originalFinalPrice,originalMetaBriteItemPrice,deleted,priceAfterCoupon,metabriteCampaignId
0,4011,ITEM NOT FOUND,26.00,26.00,False,1,True,5.0,4011,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4011,ITEM NOT FOUND,1,1,NaN,1,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,028400642255,DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCE...,10.00,10.00,True,2,True,1.0,028400642255,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,False,1,True,NaN,4011,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4011,ITEM NOT FOUND,28.00,28.00,False,1,True,4.0,4011,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7376,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7377,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7378,B076FJ92M4,mueller austria hypergrind precision electric ...,22.97,22.97,NaN,0,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.97,NaN
7379,B07BRRLSVC,thindust summer face mask - sun protection nec...,11.99,11.99,NaN,1,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.99,NaN


In [26]:
# merging the main df with results from expanded df
receipts_final = pd.merge(
    receipts,
    receipts_rewardsReceiptItemList,
    left_index=True,
    right_index=True,
    how="outer",
)

In [27]:
receipts_final.columns

Index(['_id', 'bonusPointsEarned', 'bonusPointsEarnedReason', 'createDate',
       'dateScanned', 'finishedDate', 'modifyDate', 'pointsAwardedDate',
       'pointsEarned_x', 'purchaseDate', 'purchasedItemCount',
       'rewardsReceiptItemList', 'rewardsReceiptStatus', 'totalSpent',
       'userId', 'barcode', 'description', 'finalPrice', 'itemPrice',
       'needsFetchReview', 'partnerItemId', 'preventTargetGapPoints',
       'quantityPurchased', 'userFlaggedBarcode', 'userFlaggedNewItem',
       'userFlaggedPrice', 'userFlaggedQuantity', 'needsFetchReviewReason',
       'pointsNotAwardedReason', 'pointsPayerId', 'rewardsGroup',
       'rewardsProductPartnerId', 'userFlaggedDescription',
       'originalMetaBriteBarcode', 'originalMetaBriteDescription', 'brandCode',
       'competitorRewardsGroup', 'discountedItemPrice',
       'originalReceiptItemText', 'itemNumber',
       'originalMetaBriteQuantityPurchased', 'pointsEarned_y', 'targetPrice',
       'competitiveProduct', 'originalFin

In [28]:
receipts_final

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned_x,purchaseDate,...,itemNumber,originalMetaBriteQuantityPurchased,pointsEarned_y,targetPrice,competitiveProduct,originalFinalPrice,originalMetaBriteItemPrice,deleted,priceAfterCoupon,metabriteCampaignId
0,{'$oid': '5ff1e1eb0a720f0523000575'},500.0,"Receipt number 2 completed, bonus point schedu...",{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687536000},{'$date': 1609687531000},500.0,{'$date': 1609632000000},...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,{'$oid': '5ff1e1bb0a720f052300056b'},150.0,"Receipt number 5 completed, bonus point schedu...",{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687488000},{'$date': 1609687483000},150.0,{'$date': 1609601083000},...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,{'$oid': '5ff1e1f10a720f052300057a'},5.0,All-receipts receipt bonus,{'$date': 1609687537000},{'$date': 1609687537000},NaN,{'$date': 1609687542000},NaN,5.0,{'$date': 1609632000000},...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,{'$oid': '5ff1e1ee0a7214ada100056f'},5.0,All-receipts receipt bonus,{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687539000},{'$date': 1609687534000},5.0,{'$date': 1609632000000},...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,{'$oid': '5ff1e1d20a7214ada1000561'},5.0,All-receipts receipt bonus,{'$date': 1609687506000},{'$date': 1609687506000},{'$date': 1609687511000},{'$date': 1609687511000},{'$date': 1609687506000},5.0,{'$date': 1609601106000},...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7376,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7377,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.97,NaN
7379,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.99,NaN


In [29]:
# normalizing the rest of the date columns
receipts_final_id = json_normalize(receipts_final, "_id")
receipts_final_createDate = json_normalize(receipts_final, "createDate")
receipts_final_dateScanned = json_normalize(receipts_final, "dateScanned")
receipts_final_finishedDate = json_normalize(receipts_final, "finishedDate")
receipts_final_modifyDate = json_normalize(receipts_final, "modifyDate")
receipts_final_pointsAwardedDate = json_normalize(receipts_final, "pointsAwardedDate")
receipts_final_purchaseDate = json_normalize(receipts_final, "purchaseDate")

In [30]:
receipts_final_purchaseDate

,$date
0,1.609632e+12
1,1.609601e+12
2,1.609632e+12
3,1.609632e+12
4,1.609601e+12
...,...
7376,NaN
7377,NaN
7378,NaN
7379,NaN


In [31]:
# replacing the old dictioanry columns with new cleand values
receipts_final["_id"] = receipts_final_id["$oid"]
receipts_final["createDate"] = receipts_final_createDate["$date"]
receipts_final["dateScanned"] = receipts_final_dateScanned["$date"]
receipts_final["finishedDate"] = receipts_final_finishedDate["$date"]
receipts_final["modifyDate"] = receipts_final_modifyDate["$date"]
receipts_final["pointsAwardedDate"] = receipts_final_pointsAwardedDate["$date"]
receipts_final["purchaseDate"] = receipts_final_purchaseDate["$date"]

In [32]:
# changing the date from epoch to date time format the epoch in the data is in mili seconds
receipts_final["createDate"] = pd.to_datetime(receipts_final["createDate"], unit="ms")
receipts_final["dateScanned"] = pd.to_datetime(receipts_final["dateScanned"], unit="ms")
receipts_final["finishedDate"] = pd.to_datetime(
    receipts_final["finishedDate"], unit="ms"
)
receipts_final["modifyDate"] = pd.to_datetime(receipts_final["modifyDate"], unit="ms")
receipts_final["pointsAwardedDate"] = pd.to_datetime(
    receipts_final["pointsAwardedDate"], unit="ms"
)
receipts_final["purchaseDate"] = pd.to_datetime(
    receipts_final["purchaseDate"], unit="ms"
)

In [33]:
receipts_final.drop(["rewardsReceiptItemList"], axis=1, inplace=True)

In [34]:
receipts_final.head()

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned_x,purchaseDate,...,itemNumber,originalMetaBriteQuantityPurchased,pointsEarned_y,targetPrice,competitiveProduct,originalFinalPrice,originalMetaBriteItemPrice,deleted,priceAfterCoupon,metabriteCampaignId
0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedu...",2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:36,2021-01-03 15:25:31,500.0,2021-01-03 00:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:48,2021-01-03 15:24:43,150.0,2021-01-02 15:24:43,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5ff1e1f10a720f052300057a,5.0,All-receipts receipt bonus,2021-01-03 15:25:37,2021-01-03 15:25:37,NaT,2021-01-03 15:25:42,NaT,5.0,2021-01-03 00:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5ff1e1ee0a7214ada100056f,5.0,All-receipts receipt bonus,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:39,2021-01-03 15:25:34,5.0,2021-01-03 00:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5ff1e1d20a7214ada1000561,5.0,All-receipts receipt bonus,2021-01-03 15:25:06,2021-01-03 15:25:06,2021-01-03 15:25:11,2021-01-03 15:25:11,2021-01-03 15:25:06,5.0,2021-01-02 15:25:06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
receipts_final.shape

(7381, 48)

In [36]:
# checking for duplicates
receipts_dup = receipts_final.drop_duplicates(
    subset=None, keep="first", inplace=False, ignore_index=False
)
print(receipts_dup.shape)

(6524, 48)


approx 11% of the entries are duplicates

## Users Data Exploration and data cleaning

In [37]:
users = pd.read_json("users.json.gz", lines=True, compression="gzip")
users.head()

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
1,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
2,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
3,{'$oid': '5ff1e1eacfcf6c399c274ae6'},True,{'$date': 1609687530554},{'$date': 1609687530597},consumer,Email,WI
4,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI


In [38]:
users.shape

(495, 7)

In [39]:
users.isnull().sum(axis=0)

_id              0
active           0
createdDate      0
lastLogin       62
role             0
signUpSource    48
state           56
dtype: int64

In [40]:
users.columns

Index(['_id', 'active', 'createdDate', 'lastLogin', 'role', 'signUpSource',
       'state'],
      dtype='object')

In [41]:
users_id = json_normalize(users, "_id")
users_createdDate = json_normalize(users, "createdDate")
users_lastLogin = json_normalize(users, "lastLogin")

In [42]:
users["_id"] = users_id["$oid"]
users["createdDate"] = users_createdDate["$date"]
users["lastLogin"] = users_lastLogin["$date"]

In [43]:
users

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,5ff1e194b6a9d73a3a9f1052,True,1609687444800,1.609688e+12,consumer,Email,WI
1,5ff1e194b6a9d73a3a9f1052,True,1609687444800,1.609688e+12,consumer,Email,WI
2,5ff1e194b6a9d73a3a9f1052,True,1609687444800,1.609688e+12,consumer,Email,WI
3,5ff1e1eacfcf6c399c274ae6,True,1609687530554,1.609688e+12,consumer,Email,WI
4,5ff1e194b6a9d73a3a9f1052,True,1609687444800,1.609688e+12,consumer,Email,WI
...,...,...,...,...,...,...,...
490,54943462e4b07e684157a532,True,1418998882381,1.614963e+12,fetch-staff,NaN,NaN
491,54943462e4b07e684157a532,True,1418998882381,1.614963e+12,fetch-staff,NaN,NaN
492,54943462e4b07e684157a532,True,1418998882381,1.614963e+12,fetch-staff,NaN,NaN
493,54943462e4b07e684157a532,True,1418998882381,1.614963e+12,fetch-staff,NaN,NaN


In [44]:
# since epoch time was in mili seconds
users["createdDate"] = pd.to_datetime(users["createdDate"], unit="ms")

In [45]:
users["lastLogin"] = pd.to_datetime(users["lastLogin"], unit="ms")
users.head()

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04.800,2021-01-03 15:25:37.857999872,consumer,Email,WI
1,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04.800,2021-01-03 15:25:37.857999872,consumer,Email,WI
2,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04.800,2021-01-03 15:25:37.857999872,consumer,Email,WI
3,5ff1e1eacfcf6c399c274ae6,True,2021-01-03 15:25:30.554,2021-01-03 15:25:30.596999936,consumer,Email,WI
4,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04.800,2021-01-03 15:25:37.857999872,consumer,Email,WI


In [46]:
users.dtypes

_id                     object
active                    bool
createdDate     datetime64[ns]
lastLogin       datetime64[ns]
role                    object
signUpSource            object
state                   object
dtype: object

In [47]:
users.shape

(495, 7)

In [48]:
# checking for duplicates
users_dup = users.drop_duplicates(
    subset=None, keep="first", inplace=False, ignore_index=False
)
print(users_dup.shape)

(212, 7)


In [49]:
495-212

283

approx 57% of users entries are duplicates

In [50]:
## To Run SQL querries in python using pandas dataframe use the below code answers to the querries can be found in the solution pdf

In [51]:
!pip3 install pandasql
import pandasql as ps

In [52]:
## just an example to run sql querries

In [53]:
q1 = """SELECT _id FROM brands limit 2 """

In [54]:
print(ps.sqldf(q1, locals()))

                        _id
0  601ac115be37ce2ead437551
1  601c5460be37ce2ead43755f


In [55]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 495 entries, 0 to 494
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   _id           495 non-null    object        
 1   active        495 non-null    bool          
 2   createdDate   495 non-null    datetime64[ns]
 3   lastLogin     433 non-null    datetime64[ns]
 4   role          495 non-null    object        
 5   signUpSource  447 non-null    object        
 6   state         439 non-null    object        
dtypes: bool(1), datetime64[ns](2), object(4)
memory usage: 23.8+ KB


In [56]:
brands.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1167 entries, 0 to 1166
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   _id           1167 non-null   object 
 1   barcode       1167 non-null   int64  
 2   category      1012 non-null   object 
 3   categoryCode  517 non-null    object 
 4   name          1167 non-null   object 
 5   topBrand      555 non-null    float64
 6   brandCode     933 non-null    object 
 7   cpg_ref       1167 non-null   object 
 8   cpg_id_oid    1167 non-null   object 
dtypes: float64(1), int64(1), object(7)
memory usage: 82.2+ KB


In [57]:
receipts_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7381 entries, 0 to 7380
Data columns (total 48 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   _id                                 1119 non-null   object        
 1   bonusPointsEarned                   544 non-null    float64       
 2   bonusPointsEarnedReason             544 non-null    object        
 3   createDate                          1119 non-null   datetime64[ns]
 4   dateScanned                         1119 non-null   datetime64[ns]
 5   finishedDate                        568 non-null    datetime64[ns]
 6   modifyDate                          1119 non-null   datetime64[ns]
 7   pointsAwardedDate                   537 non-null    datetime64[ns]
 8   pointsEarned_x                      609 non-null    float64       
 9   purchaseDate                        671 non-null    datetime64[ns]
 10  purchasedItemCount      